# Requirements

In [1]:
%pip install transformers
%pip install torch
%pip install pickle5
%pip install mpld3
%pip install scikit-learn
%pip install pattern3
%pip install nltk
%pip install pattern

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Load pretrained Bert model and tokenizer

In [20]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# debiased_model = "positional"
# folder_num = "5."
# model_path="bert-base-uncased"
# model_path = "../debiased_models/sent_debias/debias-BERT/experiments/acl2020-results/QNLI/debiased_final_final"
# model = BertModel.from_pretrained(model_path, output_hidden_states = True)



# debiased_model = "sent_debiased"
# folder_num = "2."
# model_path = "../debiased_models/sent_debias/debias-BERT/experiments/acl2020-results/QNLI/debiased_final_final"
# model = BertModel.from_pretrained(model_path, output_hidden_states = True)

# debiased_model = "contextualised"
# folder_num = "3."
# model_path = "../debiased_models/contextualised-embeddings-bert"
# model = BertModel.from_pretrained(model_path, output_hidden_states = True)

debiased_model = "cds"
folder_num = "4."
model_path = "../debiased_models/cds.pt"
model = BertForMaskedLM.from_pretrained('bert-base-uncased',
                                        output_attentions = False,
                                        output_hidden_states = True)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

save_path = f"../data/extracted_positional/{folder_num} {debiased_model}/{debiased_model}_sentence_"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
SINGULAR_NOUN_TEMPLATES = (
    'This is {article} {term}.',
    'That is {article} {term}.',
    'There is {article} {term}.',
    'Here is {article} {term}.',
    'The {term} is here.',
    'The {term} is there.',
)

PLURAL_NOUN_TEMPLATES = (
    'These are {term}.',
    'Those are {term}.',
    'They are {term}.',
    'The {term} are here.',
    'The {term} are there.',
)

TARGET_POSITON=[(3, -2), (3, -2), (3, -2), (3, -2), (2, -4), (2, -4), (3, -2), (3, -2), (3, -2), (2, -4), (2, -4)]

def fill_template(template, term):
    article = (
        'an'
        if (
            (
                term.startswith('honor') or any(
                    term.startswith(c) for c in 'aeiouAEIOU'
                )
            ) and not (
                term.startswith('European') or term.startswith('Ukrainian')
            )
        )
        else 'a'
    )
    sentence = template.format(article=article, term=term)
    return sentence[0].upper() + sentence[1:]

In [13]:
from pattern.en import pluralize, singularize
import nltk

def generate_noun_sentences(vocab):
    tags = [(word, tag) for w in vocab for word, tag in nltk.pos_tag([w])]
    nouns = [word for word, tag in tqdm(tags) if tag.startswith("N") and '___' not in word and word is not "_"]
    w2i = {w: i for i, w in enumerate(nouns)}

    sentence_list = []
    for term in tqdm(nouns):
        singular_term = singularize(term)
        sentences = []
        sentences += [fill_template(template, singular_term) for template in SINGULAR_NOUN_TEMPLATES]
        plurar_term = pluralize(term)
        sentences += [fill_template(template, plurar_term) for template in PLURAL_NOUN_TEMPLATES]
        sentence_list.append(sentences)
    return w2i, nouns, sentence_list

# open file and extract bert embedding function

In [14]:
import pickle5 as pickle
from tqdm import tqdm 
import pickle5 as pickle

def dump_pklfile(file, filepath, size):
	with open(filepath, "wb") as f:
		if (size == 0):
			pickle.dump((file), f)
			return
		if (size > 0):
			pickle.dump((file[:size]), f)
			return
		else:
			pickle.dump((file[size:]), f)
			return
    
def open_pklfile(filepath, size):
	with open(filepath, "rb") as f:
		if (size == 0):
			return pickle.load(f)
		return (pickle.load(f))[0:size]

gender_target_list=["male", "males", "man", "men", "boy", "boys", "brother", "brothers", "his", "he", "him", "son", "sons",
              "female", "females", "woman", "women", "girl", "girls", "sister","sisters", "she", "her","hers", "daughter", "daughters"]

def extract_bert_embeddings(sentence_list):
	#init for stacking embeddings
	embeddings = torch.empty(0, device=device)

	for sentences in tqdm(sentence_list):
		embedding = torch.empty(0, device=device)
		for i, sentence in enumerate(sentences):
			# print(f"[{sentence}]")
			# Map the token strings to their vocabulary indeces.
			marked_text = "[CLS] " + sentence + " [SEP]"
			# print(marked_text)
			tokenized_text = tokenizer.tokenize(marked_text)
			# print(tokenized_text)
			# handling such as "wedding_dress"
			tokenized_text = [token for token in tokenized_text if token != '_']

			# Split the sentence into tokens.
			indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
			segments_ids = [1] * len(tokenized_text)

			# Convert inputs to PyTorch tensors
			tokens_tensor = torch.tensor([indexed_tokens], device=device)
			segments_tensors = torch.tensor([segments_ids], device=device)

			# Put the model in "evaluation" mode,meaning feed-forward operation.
			model.eval()

			# Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
			with torch.no_grad():
				outputs = model(tokens_tensor, segments_tensors).hidden_states
				# print(f"outputs: {outputs}")
				start_position=TARGET_POSITON[i][0]
				end_position=TARGET_POSITON[i][1]
				last_four_hidden_states = outputs[-4:]
				concated_hidden_states = torch.cat(last_four_hidden_states, dim=2)
				sentence_embedding=torch.sum(torch.stack([concated_hidden_states[:,start_position], concated_hidden_states[:,end_position-1]]), 0)
				
			embedding = torch.cat([embedding, sentence_embedding], 0)
			if torch.isnan(embedding).any():
				print(sentence)
				print(tokenized_text)
				print(sentence_embedding)
				return

		sum_embedding = torch.sum(embedding, 0)

		norm_embedding = torch.nn.functional.normalize(sum_embedding, dim=0)
		norm_embedding = torch.unsqueeze(norm_embedding, dim=-1)
		norm_embedding = torch.transpose(norm_embedding, 0, 1)


		embeddings = torch.cat([embeddings, norm_embedding], 0)

	return embeddings



In [15]:

def extract_bert_embeddings_gender(sentence_list):
	#init for stacking embeddings
	embeddings = torch.empty(0, device=device)

	for sentences in tqdm(sentence_list):
		embedding = torch.empty(0, device=device)
		for i, sentence in enumerate(sentences):
			# print(f"[{sentence}]")
			# Map the token strings to their vocabulary indeces.
			marked_text = "[CLS] " + sentence + " [SEP]"
			tokenized_text = tokenizer.tokenize(marked_text)
			# handling such as "wedding_dress"
			tokenized_text = [token for token in tokenized_text if token != '_']

			# Split the sentence into tokens.
			indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
			segments_ids = [1] * len(tokenized_text)

			# Convert inputs to PyTorch tensors
			tokens_tensor = torch.tensor([indexed_tokens], device=device)
			segments_tensors = torch.tensor([segments_ids], device=device)

			# Put the model in "evaluation" mode,meaning feed-forward operation.
			model.eval()

			# Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
			with torch.no_grad():
				outputs = model(tokens_tensor, segments_tensors).hidden_states
		
				gender_position=-1
				for idx, t in enumerate(tokenized_text):
					if t in gender_target_list:
						gender_position=idx
						break
				if gender_position==-1:
					print("error")
					print(tokenized_text)
					return 

				last_four_hidden_states = outputs[-4:]
				concated_hidden_states = torch.cat(last_four_hidden_states, dim=2)
				sentence_embedding = torch.squeeze(concated_hidden_states)[gender_position]
				sentence_embedding = torch.unsqueeze(sentence_embedding, dim=0)
    
				
			embedding = torch.cat([embedding, sentence_embedding], 0)
	
			if torch.isnan(embedding).any():
				print(sentence)
				print(tokenized_text)
				print(sentence_embedding)
				return


		sum_embedding = torch.sum(embedding, 0)
		norm_embedding = torch.nn.functional.normalize(sum_embedding, dim=0)
		norm_embedding = torch.unsqueeze(norm_embedding, dim=0)
		# norm_embedding = torch.transpose(norm_embedding, 0, 1)

		embeddings = torch.cat([embeddings, norm_embedding], 1)

	return embeddings



## Extracting Embeddings: 2016 words + sent_debiased + sentence

In [17]:
vocab = open_pklfile("../data/extracted/0. original/original_word_2016_restricted_vocab.pkl", 0)
w2i, nouns, sentence_list = generate_noun_sentences(vocab)
lim_wv = extract_bert_embeddings(sentence_list)
dump_pklfile(lim_wv, f"{save_path}2016_restricted_embeddings.pkl", 0)
print(save_path)

100%|██████████| 18445/18445 [22:15<00:00, 13.81it/s]


../data/extracted_positional/3. contextualised/contextualised_sentence_


## Extracting Embeddings: 2018 words + sent_debiased + sentence

In [16]:
vocab = open_pklfile("../data/extracted/0. original/original_word_2018_restricted_vocab.pkl", 0)
w2i, nouns, sentence_list = generate_noun_sentences(vocab)
lim_wv = extract_bert_embeddings(sentence_list)
dump_pklfile(lim_wv, f"{save_path}2018_restricted_embeddings.pkl", 0)
print(save_path)

100%|██████████| 39385/39385 [48:13<00:00, 13.61it/s]


../data/extracted_positional/3. contextualised/contextualised_sentence_


## Extracting Embeddings: bert words 2500(2016) + sent_debiased + sentence

In [13]:
vocab_male_2016 = open_pklfile("../data/extracted_positional/1. bert/bert_sentence_2016_male_2500_vocab.pkl", 0)

w2i_male, nouns_male, sentence_list_male=generate_noun_sentences(vocab_male_2016)
lim_wv_male_2016 = extract_bert_embeddings(sentence_list_male)
dump_pklfile(lim_wv_male_2016, f"{save_path}2016_male_2500_embeddings.pkl", 0)

vocab_female_2016 = open_pklfile("../data/extracted_positional/1. bert/bert_sentence_2016_female_2500_vocab.pkl", 0)
w2i_female, nouns_female, sentence_list_female=generate_noun_sentences(vocab_female_2016)
lim_wv_female_2016 = extract_bert_embeddings(sentence_list_female)
dump_pklfile(lim_wv_female_2016, f"{save_path}2016_female_2500_embeddings.pkl", 0)

100%|██████████| 2500/2500 [03:03<00:00, 13.64it/s]


## Extracting Embeddings: bert words 2500(2018) + sent_debiased + sentence

In [18]:
vocab_male_2018 = open_pklfile("../data/extracted/1. bert/bert_sentence_2018_male_2500_vocab.pkl", 0)
w2i_male, nouns_male, sentence_list_male=generate_noun_sentences(vocab_male_2018)

lim_wv_male_2018 = extract_bert_embeddings(sentence_list_male)
dump_pklfile(lim_wv_male_2018, f"{save_path}2018_male_2500_embeddings.pkl", 0)

vocab_female_2018 = open_pklfile("../data/extracted/1. bert/bert_sentence_2018_female_2500_vocab.pkl", 0)
w2i_female, nouns_female, sentence_list_female=generate_noun_sentences(vocab_female_2018)
lim_wv_female_2018 = extract_bert_embeddings(sentence_list_female)
dump_pklfile(lim_wv_female_2018, f"{save_path}2018_female_2500_embeddings.pkl", 0)

100%|██████████| 2500/2500 [03:01<00:00, 13.80it/s]


## Extracting Embeddings: word file words + sent_debiased + sentence

In [19]:
with open("../data/lists/male_sentence_file.txt", 'r') as f:
  male_sentences = [sentence.strip() for sentence in f.readlines()]
male_sentence_embs = extract_bert_embeddings_gender([male_sentences])
dump_pklfile(male_sentence_embs, f"{save_path}male_word_file_embeddings.pkl", 0)

with open("../data/lists/female_sentence_file.txt", 'r') as f:
  female_sentences = [sentence.strip() for sentence in f.readlines()]
female_sentence_embs = extract_bert_embeddings_gender([female_sentences])
dump_pklfile(female_sentence_embs, f"{save_path}female_word_file_embeddings.pkl", 0)

100%|██████████| 1/1 [00:00<00:00,  1.92it/s]
